In [ ]:
#code

import os
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from io import StringIO
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, chi2_contingency
from research_common import *

set_number_statistical_tests(52)
set_global_formatting_options(format_as_std=True, num_digits=2)
set_global_journal_options("radiology")

count_tracker = {}

pd.set_option('display.max_columns', None)

data = pd.read_csv("./data/NPDB2410.CSV", low_memory=False)
count_tracker['raw_data'] = len(data)

def load_map(map_name):
    map = pd.read_excel(f"./data/code_maps/{map_name}.xlsx")
    return {row[0]: str(row[1]).strip() for row in map.itertuples(index=False)}

#map and rename columns so they're actually useful
rename_map = {
    'SEQNO': 'record_number',
    'RECTYPE': 'coded_record_type',
    'REPTYPE': 'coded_report_type',
    'ORIGYEAR': 'case_year',
    'WORKSTAT': 'work_state',
    'LICNSTAT': 'license_state',
    'LICNFELD': 'coded_license_field',
    'PRACTAGE': 'practitioner_age',
    'GRAD': 'graduation_year_group',
    'ALGNNATR': 'coded_allegation_nature',
    'ALEGATN1': 'coded_allegation_1',
    'ALEGATN2': 'coded_allegation_2',
    'OUTCOME': 'coded_severity', 
    'PAYMENT': 'payment_amount',
    'TOTALPMT': 'total_payment_amount',
    'PAYNUMBR': 'payment_number',
    'NUMBPRSN': 'number_of_practitioners',
    'PAYTYPE': 'coded_payment_type',
    'PYRRLTNS': 'coded_payment_relation',
    'PTAGE': 'patient_age_group',
    'PTSEX': 'patient_sex',
    'PTTYPE': 'coded_patient_type',
    'AAYEAR': 'action_year',
    'AALENTYP': 'coded_action_entity',
    'TYPE': 'coded_reporting_entity_type',
    'PRACTNUM': 'anon_practitioner_number',
    'NPLICRPT': 'subject_num_licensure_reports',
    'FUNDPYMT': 'coded_fund_payment_source',
}

for col in data.columns:
    if col in rename_map:
        data.rename(columns={col: rename_map[col]}, inplace=True)
    else:
        data.drop(columns=[col], inplace=True)
count_tracker['complete data'] = len(data)

def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    processed_data = data.copy()

    cached_inflation = {}
    processed_data['payment_amount'] = processed_data['payment_amount'].str.replace('$', '').str.replace(',', '').astype(float)
    processed_data['total_payment_amount'] = processed_data['total_payment_amount'].str.replace('$', '').str.replace(',', '').astype(float)
    processed_data['payment_amount_cpi_adjusted'] = processed_data.apply(lambda row: inflate_to_target_year(row['payment_amount'], 2024, row['case_year'], cached_values=cached_inflation), axis=1)
    processed_data['total_payment_amount_cpi_adjusted'] = processed_data.apply(lambda row: inflate_to_target_year(row['total_payment_amount'], 2024, row['case_year'], cached_values=cached_inflation), axis=1)
    processed_data['suit_in_same_state'] = processed_data['work_state'].str[:2] == processed_data['license_state'].str[:2]
    processed_data['outpatient'] = processed_data['coded_patient_type'] == 'O'
    processed_data['coded_severity'].astype(float)
    processed_data['coded_severity'] = processed_data['coded_severity'].replace({ 10: np.nan })
    processed_data['severe_outcome'] = processed_data['coded_severity'] >= 6
    processed_data['settled'] = processed_data['coded_payment_type'].str.capitalize() != 'J' #everything not a judgement is a settlement, per pg 26 of the data file format
    processed_data['physician_age_above_70'] = processed_data['practitioner_age'] > 70
    processed_data['has_prior_licensure_reports'] = processed_data['subject_num_licensure_reports'] > 0
    processed_data['payment_source_is_insurance'] = np.where(
        processed_data['coded_fund_payment_source'] == 0, 1,
        np.where(processed_data['coded_fund_payment_source'] == 1, 0, np.nan)
    )
    processed_data['payment_source_is_state'] = np.where(
        processed_data['coded_fund_payment_source'] == 1, 1,
        np.where(processed_data['coded_fund_payment_source'] == 0, 0, np.nan)
    )
    processed_data['patient_sex_female'] = processed_data['patient_sex'] == 'F'
    processed_data['case_initiated_by_licensing_board'] = processed_data['coded_reporting_entity_type'] == 1
    def is_diagnostic_mistake(row):
        return str(row['coded_allegation_1']) in ["101.0", "200.0", "323.0"] or str(row['coded_allegation_2']) in ["101.0", "200.0", "323.0"]
    processed_data['diagnostic_mistake'] = processed_data.apply(is_diagnostic_mistake, axis=1)

    return processed_data

processed_data = preprocess_data(data)
count_tracker['processed_data'] = len(data)
processed_data = processed_data[processed_data['coded_record_type'].isin(['M', 'P'])] #just malpractice payments
count_tracker['malpractice_data'] = len(processed_data)

restricted_physician_data = processed_data[processed_data['coded_license_field'].isin([10, 15, 20, 25])] #just MDs and DOs
restricted_radiology_physician_data = restricted_physician_data[(restricted_physician_data['coded_allegation_1'] == 321) | (restricted_physician_data['coded_allegation_2'] == 321)] #just radiologists
assert len(restricted_physician_data) == len(restricted_physician_data['record_number'].unique()), "record_number is not unique"
restricted_non_radiology_physician_data = restricted_physician_data[~restricted_physician_data['record_number'].isin(restricted_radiology_physician_data['record_number'])] #non-radiologists
count_tracker['physician_data'] = len(restricted_physician_data)

display(Markdown(f"## All Physician Data"))
display(restricted_physician_data)

# analyze the state reimbursement data for radiologists
# reimbursement data downloaded from https://www.bls.gov/oes/current/oes_${state_code}.htm
def analyze_state_reimbursement_data():
    state_df = []
    for state_file in os.listdir("./data/state_wages/"):
        state_code = state_file.split("-")[-1].split(".html")[0].strip().upper()
        #read the contents of the file and parse with beautifulsoup
        with open(f"./data/state_wages/{state_file}", "r") as f:
            contents = f.read()
            if '29-1224' not in contents:
                continue
            
            tables = pd.read_html(StringIO(contents))
            assert len(tables) == 1
            table = tables[0]
            assert 'Occupation code' in table.columns and 'Mean hourly wage' in table.columns and 'Annual mean wage' in table.columns, table.columns
            radiologist_data = table[table['Occupation code'] == '29-1224']
            everybody_data = table[table['Occupation code'] == '00-0000']
            assert len(everybody_data) == 1 and len(radiologist_data) == 1
            everybody_data = everybody_data.iloc[0]
            radiologist_data = radiologist_data.iloc[0]
            radiologist_data['state_code'] = state_code
            radiologist_data['total_employment'] = everybody_data['Employment']
            radiologist_salary = radiologist_data['Annual mean wage']
            everybody_salary = everybody_data['Annual mean wage']
            if "$" not in radiologist_salary or "$" not in everybody_salary:
                continue
            radiologist_salary = float(radiologist_salary[1:].replace(",", ""))
            everybody_salary = float(everybody_salary[1:].replace(",", ""))
            radiologist_data['salary'] = radiologist_salary
            radiologist_data['everybody_salary'] = everybody_salary
            radiologist_data['salary_ratio'] = radiologist_salary / everybody_salary
            if "(" in radiologist_data['Employment']:
                #not provided
                continue

            #number of reports in this state
            num_reports = len(restricted_radiology_physician_data[restricted_radiology_physician_data['work_state'] == state_code])
            radiologist_data['num_reports'] = num_reports
            radiologist_data['litigiousness_quotient'] = num_reports / int(radiologist_data['Employment'])
            
            num_reports_1990_2000 = len(restricted_radiology_physician_data[(restricted_radiology_physician_data['case_year'] >= 1990) & (restricted_radiology_physician_data['case_year'] < 2000) & (restricted_radiology_physician_data['work_state'] == state_code)])
            num_reports_2000_2010 = len(restricted_radiology_physician_data[(restricted_radiology_physician_data['case_year'] >= 2000) & (restricted_radiology_physician_data['case_year'] < 2010) & (restricted_radiology_physician_data['work_state'] == state_code)])
            num_reports_2010_2020 = len(restricted_radiology_physician_data[(restricted_radiology_physician_data['case_year'] >= 2010) & (restricted_radiology_physician_data['case_year'] < 2020) & (restricted_radiology_physician_data['work_state'] == state_code)])
            num_reports_2020_2025 = len(restricted_radiology_physician_data[(restricted_radiology_physician_data['case_year'] >= 2020) & (restricted_radiology_physician_data['case_year'] < 2025) & (restricted_radiology_physician_data['work_state'] == state_code)])
            num_reports_1990_2010 = num_reports_1990_2000 + num_reports_2000_2010
            num_reports_2010_2025 = num_reports_2010_2020 + num_reports_2020_2025

            num_reports_per_year_1990_2010 = num_reports_1990_2010 / 20
            num_reports_per_year_2010_2025 = num_reports_2010_2025 / 15

            radiologist_data['litigiousness_quotient_1990_2010'] = num_reports_per_year_1990_2010 / int(radiologist_data['Employment'])
            radiologist_data['litigiousness_quotient_2010_2025'] = num_reports_per_year_2010_2025 / int(radiologist_data['Employment'])

            state_df.append(radiologist_data)

    state_df = pd.DataFrame(state_df)
    return state_df

state_data = analyze_state_reimbursement_data()
display(Markdown(f"## State Data"))
display(state_data)

In [ ]:
#code

import schemdraw
from schemdraw import flow

def create_count_tracker_figure():
    with schemdraw.Drawing() as d:
        d.config(fontsize=10)
        
        complete_data = flow.Box().label(f'NPDB case log\nn={count_tracker['complete data']:,d}')
        
        arrow3 = flow.Arrow().right().at(complete_data.E)
        exclude_non_malpractice = flow.Box().label(f'Non-malpractice cases excluded\nn={count_tracker["complete data"] - count_tracker["malpractice_data"]:,d}')

        arrow4 = flow.Arrow().down().at(complete_data.S)
        malpractice_data = flow.Box().label(f'Malpractice cases\nn={count_tracker["malpractice_data"]:,d}')

        arrow5 = flow.Arrow().right().at(malpractice_data.E)
        exclude_non_physician = flow.Box().label(f'Non-physician cases excluded\nn={count_tracker["malpractice_data"] - count_tracker["physician_data"]:,d}')

        arrow6 = flow.Arrow().down().at(malpractice_data.S)
        physician_data = flow.Box().label(f'Physician (MD, DO) cases\nn={count_tracker["physician_data"]:,d}')

create_count_tracker_figure()

In [ ]:
#code

import pandas as pd
import matplotlib.pyplot as plt
from research_common import indent_text


def analyze_common_allegations(rads: pd.DataFrame, non_rads: pd.DataFrame):
    allegations_map = load_map('allegation_codes')
    def get_top_allegations(df, col1, col2, top_n = 10):
        counts = pd.concat([df[col1], df[col2]]).value_counts()
        
        #remove rows corresponding to 321.0, which is the code for radiology allegations
        counts = counts.drop(321.0, errors='ignore')
        counts = counts / counts.sum()
        return counts.head(top_n)
    
    top_rads = get_top_allegations(rads, 'coded_allegation_1', 'coded_allegation_2')
    top_non_rads = get_top_allegations(non_rads, 'coded_allegation_1', 'coded_allegation_2')

    rads_suffix = ' '
    non_rads_suffix = ''

    allegations_x = [allegations_map[e] + rads_suffix for e in list(top_rads.index)] + [allegations_map[e] + non_rads_suffix for e in list(top_non_rads.index)]
    allegations_y = list(top_rads.values) + list(top_non_rads.values)
    allegations_colors = ['blue'] * len(top_rads) + ['red'] * len(top_non_rads)
    
    plt.figure(figsize=(10, 5))
    plt.bar(allegations_x, allegations_y, color=allegations_colors)
    plt.xticks(rotation=90)
    plt.ylabel('Proportion of reports')
    plt.title('Top 10 Allegations for Radiologists vs. Non-radiologists')

    #add a legend with the colors
    blue_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Radiologists')
    red_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Non-radiologists')
    plt.legend(handles=[blue_patch, red_patch])

    plt.show()

    allegations_df = pd.DataFrame(columns=['Allegation', 'Proportion of Reports Containing Allegation'])
    allegations_df.set_index('Allegation', inplace=True)
    allegations_df.loc['Radiologists'] = ""
    for allegation in top_rads.index:
        print(allegation, allegations_map[allegation])
        allegations_df.loc[indent_text(allegations_map[allegation])] = "%s%%" % round(top_rads[allegation] * 100, 1) 
    allegations_df.loc['Non-radiologists'] = ""
    for allegation in top_non_rads.index:
        allegations_df.loc[indent_text(allegations_map[allegation] + " ")] = "%s%%" % round(top_non_rads[allegation] * 100, 1)
    display_collapsible_dataframe(allegations_df, "Top Allegations for Radiologists vs. Non-radiologists", default_open = True)


analyze_common_allegations(restricted_radiology_physician_data, restricted_non_radiology_physician_data)

In [ ]:
#code

from research_common import *

high_paying_states = set(state_data[state_data['salary_ratio'] > state_data['salary_ratio'].median()]['state_code'])
litigious_states = set(state_data[state_data['litigiousness_quotient'] > state_data['litigiousness_quotient'].median()]['state_code'])

def format_quantitative_row(pop_1: pd.DataFrame, pop_2: pd.DataFrame, col: str, num_digits: int):
    assert col in pop_1.columns and col in pop_2.columns, f"Column {col} not found in dataframes"

    pop_1_mean = pop_1[col].mean()
    pop_1_sem = pop_1[col].sem()
    pop_2_mean = pop_2[col].mean()
    pop_2_sem = pop_2[col].sem()
    _, p_value = ttest_ind(pop_1[col].dropna(), pop_2[col].dropna())
    p_value = format_p_value(p_value)
    return [format_mean(pop_1_mean, pop_1_sem, num_digits), format_mean(pop_2_mean, pop_2_sem, num_digits), p_value]

def format_proportion_row(pop_1: pd.DataFrame, pop_2: pd.DataFrame, col: str):
    pop_1_num = pop_1[col].sum()
    pop_1_denom = len(pop_1[pop_1[col].notnull()])
    pop_2_num = pop_2[col].sum()
    pop_2_denom = len(pop_2[pop_2[col].notnull()])

    contingency_table = np.array([[pop_1_num, pop_1_denom - pop_1_num], [pop_2_num, pop_2_denom - pop_2_num]])
    try:
        _, p_value, _, _ = chi2_contingency(contingency_table)
        p_value = format_p_value(p_value)
    except ValueError:
        p_value = '-'
    return [format_as_percentage(pop_1_num, pop_1_denom), format_as_percentage(pop_2_num, pop_2_denom), p_value]

def compare_rads_vs_nonrads_suits(rads: pd.DataFrame, non_rads: pd.DataFrame):
    comparison_df = pd.DataFrame(columns=['Radiology cases', 'Non-radiology cases', 'p-value'])

    rads = rads.copy()
    non_rads = non_rads.copy()
    rads['litigious_state'] = rads['work_state'].apply(lambda x: x in litigious_states)
    non_rads['litigious_state'] = non_rads['work_state'].apply(lambda x: x in litigious_states)
    rads['high_paying_state'] = rads['work_state'].apply(lambda x: x in high_paying_states)
    non_rads['high_paying_state'] = non_rads['work_state'].apply(lambda x: x in high_paying_states)


    comparison_df.loc['Patient factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Outpatient setting')] = format_proportion_row(rads, non_rads, 'outpatient')
    comparison_df.loc[indent_text('Severe outcome')] = format_proportion_row(rads, non_rads, 'severe_outcome')
    comparison_df.loc[indent_text('Female sex')] = format_proportion_row(rads, non_rads, 'patient_sex_female')
    comparison_df.loc[indent_text('Patient age')] = format_quantitative_row(rads, non_rads, 'patient_age_group', 1)
    
    comparison_df.loc['Case factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Patient suit filed in same state')] = format_proportion_row(rads, non_rads, 'suit_in_same_state')
    comparison_df.loc[indent_text('Payment Amount (Adjusted) ($)')] = format_quantitative_row(rads, non_rads, 'payment_amount_cpi_adjusted', 0)
    comparison_df.loc[indent_text('Settled')] = format_proportion_row(rads, non_rads, 'settled')
    comparison_df.loc[indent_text('Number of practitioners')] = format_quantitative_row(rads, non_rads, 'number_of_practitioners', 1)
    
    comparison_df.loc['Physician factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Physician age > 70')] = format_proportion_row(rads, non_rads, 'physician_age_above_70')
    comparison_df.loc[indent_text('Has prior licensure reports')] = format_proportion_row(rads, non_rads, 'has_prior_licensure_reports')
    comparison_df.loc[indent_text('Payment made by insurance')] = format_proportion_row(rads, non_rads, 'payment_source_is_insurance')

    comparison_df.loc['State factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Litigious state')] = format_proportion_row(rads, non_rads, 'litigious_state')
    comparison_df.loc[indent_text('High compensation state')] = format_proportion_row(rads, non_rads, 'high_paying_state')

    display_collapsible_dataframe(comparison_df, "Comparison of radiology vs. non-radiology suits", default_open = True)

    return comparison_df


def compare_low_vs_high_payment():
    df = restricted_radiology_physician_data.copy()
    df['litigious_state'] = df['work_state'].apply(lambda x: x in litigious_states)
    df['high_paying_state'] = df['work_state'].apply(lambda x: x in high_paying_states)
    lower_payment = df[df['payment_amount'] < df['payment_amount'].median()]
    higher_payment = df[df['payment_amount'] >= df['payment_amount'].median()]

    comparison_df = pd.DataFrame(columns=['Lower payment', 'Higher payment', 'p-value '])

    comparison_df.loc['Patient factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Outpatient setting')] = format_proportion_row(lower_payment, higher_payment, 'outpatient')
    comparison_df.loc[indent_text('Severe outcome')] = format_proportion_row(lower_payment, higher_payment, 'severe_outcome')
    comparison_df.loc[indent_text('Female sex')] = format_proportion_row(lower_payment, higher_payment, 'patient_sex_female')
    comparison_df.loc[indent_text('Patient age')] = format_quantitative_row(lower_payment, higher_payment, 'patient_age_group', 1)

    comparison_df.loc['Case factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Patient suit filed in same state')] = format_proportion_row(lower_payment, higher_payment, 'suit_in_same_state')
    comparison_df.loc[indent_text('Payment Amount (Adjusted, $)')] = format_quantitative_row(lower_payment, higher_payment, 'payment_amount_cpi_adjusted', 0)

    #wipe the p-value for this row, since it's not meaningful
    comparison_df.loc[indent_text('Payment Amount (Adjusted, $)'), 'p-value '] = '-'
    comparison_df.loc[indent_text('Settled')] = format_proportion_row(lower_payment, higher_payment, 'settled')
    comparison_df.loc[indent_text('Number of practitioners')] = format_quantitative_row(lower_payment, higher_payment, 'number_of_practitioners', 1)

    comparison_df.loc['Physician factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Physician age > 70')] = format_proportion_row(lower_payment, higher_payment, 'physician_age_above_70')
    comparison_df.loc[indent_text('Has prior licensure reports')] = format_proportion_row(lower_payment, higher_payment, 'has_prior_licensure_reports')
    comparison_df.loc[indent_text('Payment made by insurance')] = format_proportion_row(lower_payment, higher_payment, 'payment_source_is_insurance')

    comparison_df.loc['State factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Litigious state')] = format_proportion_row(lower_payment, higher_payment, 'litigious_state')
    comparison_df.loc[indent_text('High compensation state')] = format_proportion_row(lower_payment, higher_payment, 'high_paying_state')

    display_collapsible_dataframe(comparison_df, "Comparison of low vs. high payment radiology suits", default_open = True)

    return comparison_df

def compare_old_vs_new_cases():
    df = restricted_radiology_physician_data.copy()
    df['litigious_state'] = df['work_state'].apply(lambda x: x in litigious_states)
    df['high_paying_state'] = df['work_state'].apply(lambda x: x in high_paying_states)
    old_cases = df[df['case_year'] < 2010]
    new_cases = df[df['case_year'] >= 2010]

    comparison_df = pd.DataFrame(columns=['Case year 1990-2010', 'Case year 2010-2025', 'p-value  '])

    comparison_df.loc['Patient factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Outpatient setting')] = format_proportion_row(old_cases, new_cases, 'outpatient')
    comparison_df.loc[indent_text('Severe outcome')] = format_proportion_row(old_cases, new_cases, 'severe_outcome')
    comparison_df.loc[indent_text('Female sex')] = format_proportion_row(old_cases, new_cases, 'patient_sex_female')
    comparison_df.loc[indent_text('Patient age')] = format_quantitative_row(old_cases, new_cases, 'patient_age_group', 1)

    comparison_df.loc['Case factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Patient suit filed in same state')] = format_proportion_row(old_cases, new_cases, 'suit_in_same_state')
    comparison_df.loc[indent_text('Settled')] = format_proportion_row(old_cases, new_cases, 'settled')
    comparison_df.loc[indent_text('Payment Amount (Adjusted, $)')] = format_quantitative_row(old_cases, new_cases, 'payment_amount_cpi_adjusted', 0)
    comparison_df.loc[indent_text('Number of practitioners')] = format_quantitative_row(old_cases, new_cases, 'number_of_practitioners', 1)

    comparison_df.loc['Physician factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Physician age > 70')] = format_proportion_row(old_cases, new_cases, 'physician_age_above_70')
    comparison_df.loc[indent_text('Has prior licensure reports')] = format_proportion_row(old_cases, new_cases, 'has_prior_licensure_reports')
    comparison_df.loc[indent_text('Payment made by insurance')] = format_proportion_row(old_cases, new_cases, 'payment_source_is_insurance')

    comparison_df.loc['State factors'] = ["", "", ""]
    comparison_df.loc[indent_text('Litigious state')] = format_proportion_row(old_cases, new_cases, 'litigious_state')
    comparison_df.loc[indent_text('High compensation state')] = format_proportion_row(old_cases, new_cases, 'high_paying_state')

    display_collapsible_dataframe(comparison_df, "Comparison of old vs. new cases", default_open = True)

    return comparison_df

comparison_df_rads_vs_nonrads = compare_rads_vs_nonrads_suits(restricted_radiology_physician_data, restricted_non_radiology_physician_data)
comparison_df_low_vs_high_payment = compare_low_vs_high_payment()
comparison_df_old_vs_new_cases = compare_old_vs_new_cases()

In [ ]:
import plotly.express as px
from PIL import Image
from IPython.display import Image


states_list = []
colors = []
for row_idx, row in state_data.iterrows():
    states_list.append(row['state_code'])
    if row['state_code'] in high_paying_states and row['state_code'] in litigious_states:
        colors.append('High compensating, More litigious')
    elif row['state_code'] in high_paying_states and row['state_code'] not in litigious_states:
        colors.append('High compensating, Less litigious')
    elif row['state_code'] not in high_paying_states and row['state_code'] in litigious_states:
        colors.append('Low compensating, More litigious')
    elif row['state_code'] not in high_paying_states and row['state_code'] not in litigious_states:
        colors.append('Low compensating, Less litigious')
    else:
        colors.append('Unknown')

for state in states_list:
    if state not in states_list:
        states_list.append(state)
        colors.append('Unknown')

state_classifications = pd.DataFrame({
    'state_code': states_list,
    'state_category': colors
})

fig = px.choropleth(
    locations=states_list,
    locationmode="USA-states",
    color=colors,
    scope="usa",
    labels={"color": "State Category"},
    color_discrete_map={
        'High compensating, More litigious': 'orange',
        'High compensating, Less litigious': 'green',
        'Low compensating, More litigious': 'red',
        'Low compensating, Less litigious': 'yellow',
        'Unknown': 'lightgray'
    },
    title="State Compensation and Litigiousness for Radiology",
)
fig.show()

display(state_classifications['state_category'].value_counts())
display(Image(filename='./notebook_output/state_map_v2.png'))

In [ ]:
#perform a multivariate analysis to predict payment amount

import statsmodels.api as sm
import forestplot as fp
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import myforestplot as mfp

assert len(litigious_states) > 0 and len(high_paying_states) > 0, "Litigious states or high paying states not defined"

def perform_multivariate_analysis(df: pd.DataFrame):
    df = df.copy()
    df = df[df['payment_amount_cpi_adjusted'].notnull()]
    df = df[df['payment_amount_cpi_adjusted'] > 0]
    df['log_payment_amount'] = np.log(df['payment_amount_cpi_adjusted'])
    df['case_year_centered'] = df['case_year'] - df['case_year'].mean()
    df['litigious_state'] = df['work_state'].apply(lambda x: x in litigious_states)
    df['high_paying_state'] = df['work_state'].apply(lambda x: x in high_paying_states)

    predictors = {
        'outpatient': 'Outpatient setting',
        'severe_outcome': 'Severe outcome',
        'patient_age_group': 'Patient age',
        'patient_sex_female': 'Patient sex (female)',
        'number_of_practitioners': 'Number of practitioners',
        'physician_age_above_70': 'Physician age > 70',
        'has_prior_licensure_reports': 'Has prior licensure reports',
        'payment_source_is_insurance': 'Payment made by insurance',
        'litigious_state': 'Litigious state',
        'high_paying_state': 'High compensation state',
        'case_year_centered': 'Case year',
        'suit_in_same_state': 'Suit filed in same state',
    }


    X = df[predictors.keys()]
    # Normalize continuous variables
    for col in X.columns:
        if col in ['number_of_practitioners', 'case_year_centered']:
            X[col] = (X[col] - X[col].mean()) / X[col].std()

    X = sm.add_constant(X)
    for col in X.columns:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(float)
        if col in ['patient_age_group', 'number_of_practitioners', 'case_year_centered']:
            X[col] = X[col].astype(float).fillna(X[col].median())
        assert X[col].notnull().all(), f"Column {col} contains null values"
        assert np.isfinite(X[col]).all(), f"Column {col} contains infinite values"

    median_payment = df['payment_amount_cpi_adjusted'].median()
    df['high_payment'] = df['payment_amount_cpi_adjusted'] >= median_payment
    y = df['high_payment'].astype(float)
    model = sm.Logit(y, X).fit()
    # print(model.summary())

    #plot a ROC curve for this model
    y_pred = model.predict(X)
    fpr, tpr, thresholds = roc_curve(y, y_pred)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='black', label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='grey', linestyle='--', label='Random model')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

    #plot a precision-recall curve for this model
    precision, recall, thresholds = precision_recall_curve(y, y_pred)
    pr_auc = auc(recall, precision)
    plt.figure(figsize=(8, 8))
    plt.plot(recall, precision, color='black', label='Precision-Recall curve (AUC = %0.2f)' % pr_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

    #create a forest plot of the odds ratios
    params = model.params
    conf = model.conf_int()
    conf.columns = ['2.5%', '97.5%']
    odds_ratios = np.exp(params)
    conf['OR'] = np.exp(params)
    conf['OR_lower'] = np.exp(conf['2.5%'])
    conf['OR_upper'] = np.exp(conf['97.5%'])
    or_table = pd.DataFrame({
        'Odds Ratio': odds_ratios,
        'CI Lower': conf['OR_lower'],
        'CI Upper': conf['OR_upper'],
        'Log Odds': params,
        'Abs Log Odds': np.abs(params),
        'CI Log Lower': conf['2.5%'],
        'CI Log Upper': conf['97.5%'],
        'Log Odds ratio with 95% CI': params.apply(lambda x: f"{x:.3f}") + " (" + conf['2.5%'].apply(lambda x: f"{x:.3f}") + ", " + conf['97.5%'].apply(lambda x: f"{x:.3f}") + ")",
        'P-value': model.pvalues,
    })
    or_table = or_table.drop(index='const')
    or_table.index.name = 'Variable'
    or_table = or_table.reset_index()
    or_table['Variable'] = or_table['Variable'].map(predictors)
    or_table['P-value'] = or_table['P-value'].apply(format_p_value)
    or_table = or_table.sort_values(by='Log Odds', ascending=False)

    fig, ax = plt.subplots()
    fp.forestplot(
        or_table,
        estimate='Log Odds',
        ll='CI Log Lower',
        hl='CI Log Upper',
        varlabel='Variable',
        xlabel='Log Odds Ratio',
        xticks=[-3, -2, -1, 0, 1, 2, 3],
        annote=['Log Odds ratio with 95% CI'],
        annoteheaders=['Log Odds Ratio (95% CI)'],
        rightannote=['P-value'],
        right_annoteheaders=['P-value'],
        table=True,
        flush=True,
        ax=ax,
        **{
                "marker": "D",
                "markersize": 35,
                "color": "black",
                "xlinestyle": (0, (10, 5)),
                "xlinecolor": "#808080",
                "xtick_size": 12,
            },
    )

    fig.set_figheight(1.7 * fig.get_figheight())
    plt.tight_layout()
    plt.savefig('./notebook_output/multivariate_analysis_forest_plot.png', dpi=300)
    plt.show()

perform_multivariate_analysis(restricted_radiology_physician_data)

In [ ]:
from scipy.stats import ttest_rel

#Show change in litigiousness quotient over time
sorted_state_data = state_data.sort_values(by='litigiousness_quotient_1990_2010', ascending=True)
state_code_map = load_map('state_codes')
sorted_state_data['state_name'] = sorted_state_data['state_code'].apply(lambda x: state_code_map.get(x, x))
display(sorted_state_data)
_, p_value = ttest_rel(
    sorted_state_data['litigiousness_quotient_1990_2010'],
    sorted_state_data['litigiousness_quotient_2010_2025']
)
p_value = format_p_value(p_value)
plt.figure(figsize=(15, 10))
plt.scatter(sorted_state_data['litigiousness_quotient_1990_2010'], sorted_state_data['state_name'], color='green', label='1990-2010', marker='o')
plt.scatter(sorted_state_data['litigiousness_quotient_2010_2025'], sorted_state_data['state_name'], color='red', label='2010-2025', marker='s')
for i, txt in enumerate(sorted_state_data['state_name']):
    plt.plot(
        [sorted_state_data['litigiousness_quotient_1990_2010'].iloc[i], sorted_state_data['litigiousness_quotient_2010_2025'].iloc[i]],
        [txt, txt],
        color='black',
        linestyle='--',
        alpha=0.5
    )
plt.xlabel('Malpractice Cases per Radiologist-Year')
plt.ylabel('State')
plt.title('Malpractice Cases Per Radiologist-Year Over Time')
plt.grid(False)
plt.legend()
plt.text(0.94, 0.03, f'p = {p_value}', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, fontsize=12, color='black')
plt.show()

print(f"p-value for Litigiousness over time: {p_value}")

In [ ]:
missing_data_summary = []
for col in restricted_physician_data.columns:
    missing_values = restricted_physician_data[col].isnull().sum()
    total_values = len(restricted_physician_data)
    if missing_values == 0:
        continue
    missing_data_summary.append({
        'Column': " ".join(col.split("_")).title(),
        'Missing Values': format_quantity_with_percentage(missing_values, total_values, num_digits=0)
    })

missing_data_summary = pd.DataFrame(missing_data_summary)
missing_data_summary = missing_data_summary.sort_values(by='Missing Values', key=lambda x: x.str.extract(r'(\d+)', expand=False).astype(int), ascending=False)

display_collapsible_dataframe(missing_data_summary, "Missing Data Summary for Radiology Malpractice Dataset", default_open = True)